# Scraper institutes

This notebook is a monthly scraper used to retrieve information about condition in detention centers in Italy. To do so, it uses the id numbers of the various detention centers to navigate to the dedicated webpages with Selenium, store locally the html code of the page and then parse it using BeautifuSoup. The information is then stored in a pandas dataframe and saved as a csv file.

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from pathlib import Path
import datetime
from random import randint
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By

In [2]:
# Collect institutes id numbers
prisons_df = pd.read_csv('../outputs/clean/istituti_penitenziari.csv')
prison_ids = prisons_df['id_istituto'].tolist()

# Get current month
current_month = datetime.datetime.now().strftime("%Y-%m")


In [3]:
def open_browser():
    """
    Opens a new automated browser window with all tell-tales of automated browser disabled
    """
    options = Options()
    options.add_argument("--start-maximized")

    # Enable headless mode
    # options.add_argument    
    options.add_argument("--headless")

    # remove all signs of this being an automated browser
    options.set_preference("dom.webdriver.enabled", False)
    options.set_preference('useAutomationExtension', False)
    options.set_preference("general.useragent.override", "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0")
    
    return options

def get_html(prison_id):
    # open the browser with the new options
    driver = webdriver.Firefox(options=open_browser())
    
    MAX_RETRIES = 5

    for i in range(MAX_RETRIES):
        try:
            BASE_URL = "https://www.giustizia.it/giustizia/it/dettaglio_scheda.page?s="
            page_url = BASE_URL + prison_id
            print("Fetching " + page_url)

            # Go to the page, get source code
            driver.get(page_url)
            sleep(5)
            page_html = driver.page_source
            return page_html

        except Exception as e:
            print(f"Error occurred: {e}. Retrying in 10 seconds.")
            sleep(10)
        finally:
            driver.quit()


In [4]:
prison_all_data = []

for prison_id in prison_ids:

    dest = Path(f"../outputs/raw/istituti/{prison_id}_{current_month}.html")

    if dest.exists() : #... load it from file
        print(f"Already have {dest}, loading!")
        page_html = open(dest).read()
    else:
        page_html = get_html(prison_id)

    # Parse the html with BeautifulSoup
    soup = BeautifulSoup(page_html, 'html.parser')

    # Nome istituto, tipo istituto
    institute_name = soup.find('h1', {'class': 'titoloIstituto'}).text.strip()
    institute_type = soup.find('h3', {'class': 'titoloIstituto'}).text.strip()

    # Tabella capienze
    table_capienze = soup.find_all('table')[0]
    posti_regolamentari = int(table_capienze.find_all('td')[0].text.strip())
    posti_non_disponibili = int(table_capienze.find_all('td')[1].text.strip())
    totale_detenuti = int(table_capienze.find_all('td')[2].text.strip())
    
    # Dati capienze aggiornati al
    target_span= soup.find('h2', text='dati aggiornati al ')
    span = target_span.find_next_sibling('span')
    dati_aggiornati_al = span.text.strip()

    # Responsabile ASL per il carcere
    target_span = soup.find('h2', text='Responsabile ASL per il carcere')
    try:
        div = target_span.find_next('div', {'class': 'listaContenutiComplessi'})
        spans = div.find_all('span', {'class': 'valoreSottocampo'})
        asl = spans[0].text.strip()
        first_name_asl = spans[1].text.strip()
        last_name_asl = spans[2].text.strip()
    except:
        asl = 'NA'
        first_name_asl = 'NA'
        last_name_asl = 'NA'

     # Direttore
    target_span = soup.find('h2', text='Direttore')
    try:
        div = target_span.find_next('div', {'class': 'listaContenutiComplessi'})
        spans = div.find_all('span', {'class': 'valoreSottocampo'})
        first_name = spans[0].text.strip()
        last_name = spans[1].text.strip()
        role = spans[2].text.strip()
    except:
        first_name = 'NA'
        last_name = 'NA'
        role = 'NA'
    
    # Staff table
    table_staff = soup.find_all('table')[1]
    polizia_penitenziaria_effettivi = int(table_staff.find_all('td')[0].text.strip())
    polizia_penitenziaria_previsti = int(table_staff.find_all('td')[1].text.strip())
    amministrativi_effettivi = int(table_staff.find_all('td')[2].text.strip())
    amministrativi_previsti = int(table_staff.find_all('td')[3].text.strip())
    educatori_effettivi = int(table_staff.find_all('td')[4].text.strip())
    educatori_previsti = int(table_staff.find_all('td')[5].text.strip())

    # Personale polizia penitenziaria aggiornato al
    target_span= soup.find('h2', text='personale polizia penitenziaria aggiornato al')
    try:
        span = target_span.find_next_sibling('span')
        personale_polizia_aggiornato_al = span.text.strip()
    except:
        personale_polizia_aggiornato_al = 'NA'

    # Personale amministrativo aggiornato al
    target_span= soup.find('h2', text='personale amministrativo aggiornato al')
    try:
        span = target_span.find_next_sibling('span')
        personale_amministrativo_aggiornato_al = span.text.strip()
    except:
        personale_amministrativo_aggiornato_al = 'NA'


    
    prison_data = [
        [
        prison_id,
        institute_name,
        institute_type,
        posti_regolamentari,
        posti_non_disponibili,
        totale_detenuti,
        dati_aggiornati_al,
        asl,
        first_name_asl,
        last_name_asl,
        first_name,
        last_name,
        role,
        polizia_penitenziaria_effettivi,
        polizia_penitenziaria_previsti,
        amministrativi_effettivi,
        amministrativi_previsti,
        educatori_effettivi,
        educatori_previsti,
        personale_polizia_aggiornato_al,
        personale_amministrativo_aggiornato_al
        ]
        ]

    prison_all_data.extend(prison_data)

    print("#########")

prison_headers = [
    'prison_id',
    'institute_name',
    'institute_type',
    'posti_regolamentari',
    'posti_non_disponibili',
    'totale_detenuti',
    'dati_aggiornati_al',
    'asl',
    'nome_responsabile_asl',
    'cognome_responsabile_asl',
    'nome_direttore',
    'cognome_direttore',
    'ruolo_direttore',
    'polizia_penitenziaria_effettivi',
    'polizia_penitenziaria_previsti',
    'amministrativi_effettiv',
    'amministrativi_previst',
    'educatori_effettivi',
    'educatori_previsti',
    'personale_polizia_aggiornato_a',
    'personale_amministrativo_aggiornato_al'

]

prison_data_df = pd.DataFrame(prison_all_data, columns=prison_headers)

prison_data_df.to_csv('../outputs/clean/istituti_penitenziari_data.csv', index=False, encoding="UTF-8-sig")


Already have ../outputs/raw/istituti/MII179988_2024-03.html, loading!
#########
Already have ../outputs/raw/istituti/MII172610_2024-03.html, loading!
#########
Already have ../outputs/raw/istituti/MII172827_2024-03.html, loading!
#########
Already have ../outputs/raw/istituti/MII172320_2024-03.html, loading!
#########
Already have ../outputs/raw/istituti/MII173712_2024-03.html, loading!
#########
Already have ../outputs/raw/istituti/MII173747_2024-03.html, loading!
#########
Already have ../outputs/raw/istituti/MII177436_2024-03.html, loading!
#########
Already have ../outputs/raw/istituti/MII178027_2024-03.html, loading!
#########
Already have ../outputs/raw/istituti/MII178072_2024-03.html, loading!
#########
Already have ../outputs/raw/istituti/MII178659_2024-03.html, loading!
#########
Already have ../outputs/raw/istituti/MII179237_2024-03.html, loading!
#########
Already have ../outputs/raw/istituti/MII182910_2024-03.html, loading!
#########
Already have ../outputs/raw/istituti/MII